In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
from sklearn import svm
from sklearn import mixture
from sklearn.metrics import confusion_matrix
import glob
from sklearn.decomposition import PCA

In [10]:
abdullah_training_directory = './images/Abdullah/training/'
mustafa_training_directory = './images/Mustafa/training/'
saleh_training_directory = './images/Saleh/training/'
adham_training_directory = './images/Adham/training/'

abdullah_testing_directory = './images/Abdullah/testing/'
mustafa_testing_directory = './images/Mustafa/testing/'
saleh_testing_directory = './images/Saleh/testing/'
adham_testing_directory = './images/Adham/testing/'


training_images_directories = [abdullah_training_directory, mustafa_training_directory, saleh_training_directory, adham_training_directory]
testing_images_directories = [abdullah_testing_directory, mustafa_testing_directory, saleh_testing_directory, adham_testing_directory]

training_images = []
testing_images = []

# Load the cascade for facial detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Populate Training Image list
for directory in training_images_directories:
    images = [cv2.imread(file) for file in glob.glob("{directory}*.jpg".format(directory=directory))]
    for img in images:
        training_images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
# Populate Testing Image list
for directory in testing_images_directories:
    images = [cv2.imread(file) for file in glob.glob("{directory}*.jpg".format(directory=directory))]
    for img in images:
        testing_images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

"""
4.1 + 4.3 --- Training Images Feature Extraction using SIFT
"""

# create a SIFT object
sift = cv2.xfeatures2d.SIFT_create()

training_bounds = []

for i in range(60):
    img = training_images[i]
    # copy image
    img_disp = img.copy()
    # convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # detect harris corners
    # cv2.cornerHarris(src, neighborhoodsize, sobelKernelSize, Harris_k)
    corners = cv2.cornerHarris(img_gray, 2, 3, 0.04)

    # normalize corner map [0,1]
    # cv2.normalize(src, dst, alpha, beta, norm_type)
    # min_I(dst(I)) = alpha, max_I(dst(I)) = beta
    cv2.normalize(corners, corners, 0, 1, cv2.NORM_MINMAX)

    # threshold for an optimal value
    thres = 0.5

    # list all points higher than threshold
    loc = np.where(corners >= thres)
    # loop though points
    
    pts = []
    
    for pt in zip(*loc[::-1]):
        pts.append(pt)
        
        # draw filled circle on each point
        cv2.circle(img_disp, pt, 4, (255,0,0), -1)

    xs, ys = np.array(list(zip(*pts)))
    bounds_dict = {"min_x" : np.amin(xs), "min_y" : np.amin(ys), "max_x" : np.amax(xs), "max_y" : np.amax(ys)}
    training_bounds.append(bounds_dict)

In [11]:
def extract_face(img, bounds):
    min_x =  bounds["min_x"]
    max_x =  bounds["max_x"]
    min_y =  bounds["min_y"]
    max_y =  bounds["max_y"]

    return img[min_y:max_y, min_x:max_x,:]

In [ ]:
plt.figure(figsize=(20, 80))
training_faces = []
for i in range(60):
    img = training_images[i]
    face = extract_face(img, training_bounds[i])
    face = cv2.resize(face, (128,128))
    training_faces.append(face)
     # display images
    plt.subplot(15, 4, i+1)
    plt.imshow(face)    

In [ ]:
X_train = np.array(training_faces).reshape(len(training_faces), -1)
pca = PCA(n_components=15)
pca.fit(X_train)

plt.imshow(pca.mean_.reshape(training_faces[0].shape).astype('uint8'), cmap='gray')
plt.title('Mean Image');


In [ ]:
fig = plt.figure(figsize=(16, 6))
for i in range(5):
    ax = fig.add_subplot(1, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(pca.components_[i].reshape(training_faces[0].shape).astype('uint8'))